# Data Preprocessing for Psych8k Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# OpenAI API Key
openai.api_key = "your_openai_api_key_here"

In [ ]:
# Step 1: Load the Psych8k dataset and inspect
import pandas as pd

# Define the dataset path
psych8k_path = "/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/Psych8k/Psych8k.csv"

# Load the dataset
psych8k_df = pd.read_csv(psych8k_path)

# Display the first few rows to understand the structure
print("Dataset Loaded Successfully!")
psych8k_df.head()

Dataset Loaded Successfully!


,input,output
0,I have been struggling with these negative voi...,The exercise we will try involves visualizing ...
1,"I don't like depression, but I've learned that...",It's good that you recognize your feelings and...
2,I felt like there was nobody to advocate for m...,It sounds like you were experiencing a sense o...
3,I can't be open with my parents as much as mos...,I understand that it might be difficult for yo...
4,I feel confused and it's quite painful. This c...,It sounds like you're experiencing pain and sa...


In [ ]:
# Step 2: Validate and clean the Psych8k dataset

# Check for missing values
missing_values = psych8k_df.isnull().sum()
print("Missing Values in Each Column:\n", missing_values)

# Drop rows with missing values
psych8k_df = psych8k_df.dropna()

# Remove duplicates
psych8k_df = psych8k_df.drop_duplicates()

# Verify dataset shape after cleaning
print(f"Dataset shape after cleaning: {psych8k_df.shape}")

Missing Values in Each Column:
 input     0
output    0
dtype: int64
Dataset shape after cleaning: (8086, 2)


In [ ]:
pip install openai==0.28

In [ ]:
# STEP 1: Install required library
# !pip install openai --quiet

# STEP 2: Import libraries
import pandas as pd
import openai
import ast
import re
import time
from tqdm import tqdm

# STEP 3: Set your OpenAI API Key (replace with your own)
openai.api_key = "sk-BQeOeWHdHJvS4uJYMkf76e0oUYgrxeZM4WNk6BNZS5T3BlbkFJEt3ksOTNYzvY5i6UIH47ykF2rS1-7lzgL0zWhzh1IA"  # <- 🔐 Set securely via environment variable in production

# STEP 4: Load only the first 100 rows for testing
psych8k_path = "/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/Psych8k/Psych8k.csv"
df = pd.read_csv(psych8k_path)

# STEP 5: Function to get cognitive distortions from GPT-4o-mini
def get_distortions_batch(batch_inputs):
    prompt = (
        "You are a mental health assistant trained in Cognitive Behavioral Therapy (CBT). "
        "For each user input below, identify any cognitive distortions present "
        "from the following list: all-or-nothing thinking, overgeneralization, mental filter, "
        "disqualifying the positive, jumping to conclusions, magnification, emotional reasoning, "
        "should statements, labeling, personalization, blaming others.\n\n"
        "Return a JSON-style Python list of strings, where each item is the distortion(s) "
        "for the corresponding input. If none are found, return \"none\". Example:\n"
        "[\"should statements\", \"none\", \"labeling, personalization\"]\n\n"
        "Inputs:\n" +
        "\n".join([f"{i+1}. {text}" for i, text in enumerate(batch_inputs)])
    )

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=1024
        )
        # extract inside code block if wrapped in ```python ... ```
        output_text = response['choices'][0]['message']['content'].strip()
        match = re.search(r"```(?:python)?\s*(\[[\s\S]*?\])\s*```", output_text)
        if match:
            output_text = match.group(1)
        return ast.literal_eval(output_text)
    except Exception as e:
        print("⚠️ Error parsing output:", e)
        return ["error"] * len(batch_inputs)

# STEP 6: Run in batches
batch_size = 20
results = []

for i in tqdm(range(0, len(df), batch_size)):
    batch = df['input'].iloc[i:i+batch_size].tolist()
    distortions = get_distortions_batch(batch)
    results.extend(distortions)
    time.sleep(3)  # <-- Added 15-second delay

# STEP 7: Add results to DataFrame
df['cognitive_distortions'] = results

# STEP 8: Save to new CSV
df.to_csv("Psych8k_with_distortions_sample.csv", index=False)

# STEP 9: Preview
df.head()


100%|██████████| 405/405 [34:09<00:00,  5.06s/it]


ValueError: Length of values (8083) does not match length of index (8086)

In [ ]:
# Assume `results` is your list of distortion predictions
# Pad it if it's shorter than the DataFrame
if len(results) < len(df):
    print(f"Patching: {len(df) - len(results)} missing values detected.")
    results += ["not_processed"] * (len(df) - len(results))
elif len(results) > len(df):
    print(f"Warning: {len(results) - len(df)} extra values. Trimming.")
    results = results[:len(df)]

# Now assign safely
df["cognitive_distortions"] = results


Patching: 3 missing values detected.


In [ ]:
df.to_csv("Psych8k_with_distortions_final.csv", index=False)


In [ ]:
df.head(50)

,input,output,cognitive_distortions
0,I have been struggling with these negative voi...,The exercise we will try involves visualizing ...,none
1,"I don't like depression, but I've learned that...",It's good that you recognize your feelings and...,emotional reasoning
2,I felt like there was nobody to advocate for m...,It sounds like you were experiencing a sense o...,none
3,I can't be open with my parents as much as mos...,I understand that it might be difficult for yo...,all-or-nothing thinking
4,I feel confused and it's quite painful. This c...,It sounds like you're experiencing pain and sa...,emotional reasoning
5,I'm worried that the fighting will continue in...,It's important to be cautious about that becau...,jumping to conclusions
6,I've always been taken care of by my family me...,It sounds like your family has been overly pro...,"labeling, personalization"
7,I used to be involved in some negative activit...,It's good that you've recognized the connectio...,blaming others
8,"I had hopes for my relationships to work well,...",It sounds like you're feeling disappointed abo...,none
9,I feel like people are pressuring me to get ma...,It's important to follow your own desires and ...,blaming others


In [ ]:
import pandas as pd
import json

# Step 1: Load the Psych8k dataset with cognitive distortions
psych8k_path = "/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/Psych8k/Psych8k_with_distortions.csv"  # Path to the CSV file
psych8k_df = pd.read_csv(psych8k_path)

# Step 4: Reformat the dataset into a standardized conversation structure
conversations = []  # To store the JSON structure for all conversations
for idx, row in psych8k_df.iterrows():
    user_content = row["input"]
    therapist_content = row["output"]
    distortions = row["cognitive_distortions"]

    # Build the conversation object
    conversation = {
        "conversation_id": f"psych8k_{idx}",  # Unique conversation ID
        "turns": [
            {"role": "user", "content": user_content},
            {"role": "therapist", "content": therapist_content}
        ],
        "metadata": {
            "source": "psych8k",
            "type": "single-turn",  # Psych8k is single-turn conversations
            "turn_count": 2,  # Each conversation has 2 turns (user + therapist)
            "cognitive_distortions": distortions.split(", ") if isinstance(distortions, str) else []  # Split distortions into a list
        }
    }
    conversations.append(conversation)

# Step 5: Save the preprocessed dataset as a JSON file
output_path = "Preprocessed_Psych8k_with_Cognitive_Distortions.json"  # Output filename
with open(output_path, "w") as json_file:
    json.dump(conversations, json_file, indent=4)

print(f"Preprocessed dataset saved to: {output_path}")

Preprocessed dataset saved to: Preprocessed_Psych8k_with_Cognitive_Distortions.json


In [ ]:
# Step 3: Reformat dataset into standardized conversation structure

def create_conversation(row, conversation_id):
    # Create a conversation object
    return {
        "conversation_id": f"psych_{conversation_id}",
        "turns": [
            {"role": "user", "content": row['input']},
            {"role": "therapist", "content": row['output']}
        ],
        "metadata": {
            "source": "psych8k",
            "type": "single-turn",
            "turn_count": 2,
            "cognitive_distortions": []  # Initialize with an empty list
        }
    }

# Apply the transformation
conversations = [create_conversation(row, idx) for idx, row in psych8k_df.iterrows()]

# Check an example conversation
print("Example of a reformatted conversation:")
print(conversations[0])

Example of a reformatted conversation:
{'conversation_id': 'psych_0', 'turns': [{'role': 'user', 'content': 'I have been struggling with these negative voices in my head for a long time, and they have become automatic. Recently, there was an incident that left me feeling very self-critical. I am willing to try a new exercise suggested by my counsellor.'}, {'role': 'therapist', 'content': "The exercise we will try involves visualizing your alter-ego in another chair, to help you examine and challenge the negative thoughts in your head by externalizing them. By focusing on the recent incident, we can delve into how you feel about it and understand what's going on inside you. Remember, it might feel strange, but this exercise has helped others in the past."}], 'metadata': {'source': 'psych8k', 'type': 'single-turn', 'turn_count': 2, 'cognitive_distortions': []}}


In [ ]:
psych8k_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8086 entries, 0 to 8085
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   input   8086 non-null   object
 1   output  8086 non-null   object
dtypes: object(2)
memory usage: 126.5+ KB


In [ ]:
# Step 5: Save the preprocessed dataset as a JSON file
import json

# Define output path
output_path = "/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/Psych8k/Preprocessed_Psych8k.json"

# Save conversations to JSON
with open(output_path, 'w') as json_file:
    json.dump(conversations, json_file, indent=4)

print(f"Preprocessed dataset saved to: {output_path}")

Preprocessed dataset saved to: /content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/Psych8k/Preprocessed_Psych8k.json


# Data Preprocessing for CBT Conversations Dataset

In [ ]:
# Step 1: Load the dataset and inspect
import json

# Define the dataset path
cbt_dataset_path = "/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/SynthDataset for FTing/cleaned_cbt_therapy_dataset.json"

# Load the dataset
with open(cbt_dataset_path, 'r') as f:
    cbt_data = json.load(f)

# Check the number of conversations and display an example
print(f"Total conversations in the dataset: {len(cbt_data)}")
print("Example of a conversation:")
print(json.dumps(cbt_data[0], indent=4))

Total conversations in the dataset: 3127
Example of a conversation:
{
    "conversations": [
        {
            "role": "user",
            "content": "I have applied for something and I'm not worried about it, which seems unusual for me. What would it look like for me if everything goes well?"
        },
        {
            "role": "therapist",
            "content": "That's an interesting perspective. It's great that you're not feeling worried, as that can often be a source of stress. Can you describe what 'everything going well' would look like for you? How would that make you feel?"
        },
        {
            "role": "user",
            "content": "If everything went well, I would get the job, and I think I'd feel proud and relieved."
        },
        {
            "role": "therapist",
            "content": "Feeling proud and relieved sounds wonderful! What do you think has changed for you this time, leading to this lack of worry? How does that make you feel about you

In [ ]:
# Step 2: Validate and clean conversations

# Remove incomplete conversations (less than 2 turns)
cbt_data = [conv for conv in cbt_data if len(conv.get('conversations', [])) >= 2]

# Standardize role labels
def standardize_roles(conversation):
    for turn in conversation['conversations']:
        if turn['role'].lower() in ['user', 'client', 'patient']:
            turn['role'] = 'user'
        elif turn['role'].lower() in ['therapist', 'counselor']:
            turn['role'] = 'therapist'
    return conversation

cbt_data = [standardize_roles(conv) for conv in cbt_data]

# Verify cleaned data
print(f"Conversations after cleaning: {len(cbt_data)}")
print("Example of a cleaned conversation:")
print(json.dumps(cbt_data[0], indent=4))

Conversations after cleaning: 3100
Example of a cleaned conversation:
{
    "conversations": [
        {
            "role": "user",
            "content": "I have applied for something and I'm not worried about it, which seems unusual for me. What would it look like for me if everything goes well?"
        },
        {
            "role": "therapist",
            "content": "That's an interesting perspective. It's great that you're not feeling worried, as that can often be a source of stress. Can you describe what 'everything going well' would look like for you? How would that make you feel?"
        },
        {
            "role": "user",
            "content": "If everything went well, I would get the job, and I think I'd feel proud and relieved."
        },
        {
            "role": "therapist",
            "content": "Feeling proud and relieved sounds wonderful! What do you think has changed for you this time, leading to this lack of worry? How does that make you feel about y

In [ ]:
# Step 3: Structure conversations into a consistent format

def structure_conversation(conversation, conversation_id):
    return {
        "conversation_id": f"cbt_{conversation_id}",
        "turns": [
            {"role": turn['role'], "content": turn['content']}
            for turn in conversation['conversations']
        ],
        "metadata": {
            "source": "cbt",
            "type": "multi-turn",
            "turn_count": len(conversation['conversations'])
        }
    }

# Apply the transformation
structured_conversations = [structure_conversation(conv, idx) for idx, conv in enumerate(cbt_data)]

# Check an example structured conversation
print("Example of a structured conversation:")
print(json.dumps(structured_conversations[0], indent=4))

Example of a structured conversation:
{
    "conversation_id": "cbt_0",
    "turns": [
        {
            "role": "user",
            "content": "I have applied for something and I'm not worried about it, which seems unusual for me. What would it look like for me if everything goes well?"
        },
        {
            "role": "therapist",
            "content": "That's an interesting perspective. It's great that you're not feeling worried, as that can often be a source of stress. Can you describe what 'everything going well' would look like for you? How would that make you feel?"
        },
        {
            "role": "user",
            "content": "If everything went well, I would get the job, and I think I'd feel proud and relieved."
        },
        {
            "role": "therapist",
            "content": "Feeling proud and relieved sounds wonderful! What do you think has changed for you this time, leading to this lack of worry? How does that make you feel about your curr

In [ ]:
# Step 4: Detect cognitive distortions and add tags
import re

# Cognitive Distortions Dictionary
cognitive_distortions = {
    "all_or_nothing_thinking": [
        "always", "never", "completely", "totally", "entirely", "perfect", "failure", "ruined", "no middle ground", "black and white"
    ],
    "overgeneralization": [
        "everyone", "nobody", "all the time", "never works out", "everywhere", "forever", "it’s bound to happen", "this always happens"
    ],
    "mental_filter": [
        "only see the negative", "focus on flaws", "ignore the positive", "nothing good", "all I notice is the bad", "all problems, no solutions"
    ],
    "disqualifying_the_positive": [
        "that doesn’t count", "it was just luck", "anyone could have done it", "I don’t deserve credit", "it’s not a big deal", "they’re just being nice"
    ],
    "jumping_to_conclusions_mind_reading": [
        "I know they think I’m a failure", "they must hate me", "they’re judging me", "everyone sees I’m worthless", "I can tell they don’t like me"
    ],
    "jumping_to_conclusions_fortune_telling": [
        "it will never work", "I’m doomed", "this is going to fail", "I just know it’s going to end badly", "I can’t see any good outcome"
    ],
    "magnification_or_minimization": [
        "it’s the end of the world", "this is a total disaster", "it’s nothing special", "downplay my success", "make a mountain out of a molehill", "it’s not that important"
    ],
    "emotional_reasoning": [
        "I feel it, so it must be true", "if I’m scared, there’s real danger", "I feel worthless, so I am worthless", "my emotions are facts", "I feel guilty, so I must have done something wrong"
    ],
    "should_statements": [
        "I should be able to handle this", "I must do better", "I have to be perfect", "I shouldn’t feel this way", "I ought to be stronger", "I shouldn’t make mistakes"
    ],
    "labeling": [
        "I’m a loser", "I’m worthless", "I’m a failure", "I’m useless", "I’m stupid", "I’m incompetent"
    ],
    "personalization": [
        "it’s all my fault", "I caused this", "I messed up everyone’s day", "I’m responsible for everything", "if only I had done better"
    ],
    "blaming_others": [
        "it’s all their fault", "they made this happen", "they ruined everything", "they are the reason I’m like this", "I wouldn’t feel this way if not for them"
    ]
}

# Function to detect cognitive distortions
def detect_cognitive_distortions(conversation):
    detected_distortions = set()
    for turn in conversation['turns']:
        if turn['role'] == 'user':  # Only analyze the user's turns
            for distortion, phrases in cognitive_distortions.items():
                if any(re.search(rf'\b{phrase}\b', turn['content'], re.IGNORECASE) for phrase in phrases):
                    detected_distortions.add(distortion)
    return list(detected_distortions)

# Add detected cognitive distortions to metadata
for conversation in structured_conversations:
    distortions = detect_cognitive_distortions(conversation)
    conversation['metadata']['cognitive_distortions'] = distortions

# Check an example with added cognitive distortion metadata
print("Example conversation with cognitive distortion tags:")
print(json.dumps(structured_conversations[0], indent=4))

Example conversation with cognitive distortion tags:
{
    "conversation_id": "cbt_0",
    "turns": [
        {
            "role": "user",
            "content": "I have applied for something and I'm not worried about it, which seems unusual for me. What would it look like for me if everything goes well?"
        },
        {
            "role": "therapist",
            "content": "That's an interesting perspective. It's great that you're not feeling worried, as that can often be a source of stress. Can you describe what 'everything going well' would look like for you? How would that make you feel?"
        },
        {
            "role": "user",
            "content": "If everything went well, I would get the job, and I think I'd feel proud and relieved."
        },
        {
            "role": "therapist",
            "content": "Feeling proud and relieved sounds wonderful! What do you think has changed for you this time, leading to this lack of worry? How does that make you feel 

In [ ]:
print(json.dumps(structured_conversations[974], indent=4))

{
    "conversation_id": "cbt_974",
    "turns": [
        {
            "role": "therapist",
            "content": "It sounds like you're navigating some difficult feelings about your current situation. On one hand, you want to find a place where you feel more like yourself, and on the other, you have concerns about safety and stability. Can you share more about what specifically makes you feel like you don\u2019t fit in the current community?"
        },
        {
            "role": "user",
            "content": "I just feel different from everyone else. It\u2019s like I\u2019m on the outside looking in, and I don\u2019t connect with people. I think moving away might help, but I worry about how my husband would feel about that."
        },
        {
            "role": "therapist",
            "content": "Feeling out of place can be really isolating. It\u2019s understandable to think that a new environment could provide a fresh start. How do you think your husband would respond if

In [ ]:
# Step 5: Save the preprocessed dataset as a JSON file

# Define output path
output_path = "/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/SynthDataset for FTing/Preprocessed_CBT_Dataset_Enriched.json"

# Save structured conversations to JSON
with open(output_path, 'w') as json_file:
    json.dump(structured_conversations, json_file, indent=4)

print(f"Preprocessed dataset saved to: {output_path}")

Preprocessed dataset saved to: /content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/SynthDataset for FTing/Preprocessed_CBT_Dataset_Enriched.json


# Dataset Merging

In [ ]:
/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/Psych8k/Psych8k_with_distortions.csv
/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/SynthDataset for FTing/Preprocessed_CBT_Dataset_Enriched.json

In [ ]:
import pandas as pd
import json

# Step 1: Load the Psych8k dataset
psych8k_path = "/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/Psych8k/Psych8k_with_distortions.csv"
psych8k_df = pd.read_csv(psych8k_path)

# Step 2: Convert Psych8k dataset into JSON format of conversations
psych8k_conversations = []
for idx, row in psych8k_df.iterrows():
    conversation = {
        "conversation_id": f"psych8k_{idx}",
        "turns": [
            {"role": "user", "content": row["input"]},
            {"role": "therapist", "content": row["output"]}
        ],
        "metadata": {
            "source": "psych8k",
            "type": "single-turn",
            "turn_count": 2,
            "cognitive_distortions": row["cognitive_distortions"].split(", ") if isinstance(row["cognitive_distortions"], str) else ["none"]
        }
    }
    psych8k_conversations.append(conversation)

# Step 3: Load the CBT dataset
cbt_path = "/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/SynthDataset for FTing/Preprocessed_CBT_Dataset_Enriched.json"
with open(cbt_path, "r") as cbt_file:
    cbt_conversations = json.load(cbt_file)

# Step 4: Validate and standardize CBT dataset
for conversation in cbt_conversations:
    # Add "none" if cognitive distortions are empty
    if not conversation["metadata"].get("cognitive_distortions"):
        conversation["metadata"]["cognitive_distortions"] = ["none"]
    # Ensure unique conversation IDs
    conversation["conversation_id"] = f"cbt_{conversation['conversation_id']}"

# Step 5: Merge the datasets
merged_conversations = psych8k_conversations + cbt_conversations

# Step 6: Save the merged dataset as a JSON file
output_path = "/content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/Psych8k_and_CBT_Conversation_Dataset.json"
with open(output_path, "w") as output_file:
    json.dump(merged_conversations, output_file, indent=4)

print(f"Merged dataset saved to: {output_path}")

Merged dataset saved to: /content/drive/MyDrive/ACADEMICS/FYP/4. Implementation/Datasets/Psych8k_and_CBT_Conversation_Dataset.json


In [ ]:
print(merged_conversations[500])

{'conversation_id': 'psych8k_500', 'turns': [{'role': 'user', 'content': "I'm feeling similar to when Sharon went away, and now that we won't see each other every week, I feel like I am out of the phase of the earth."}, {'role': 'therapist', 'content': "It's normal to feel a sense of loss when there's a change in the frequency of our meetings, but remember, we can still meet in three months and schedule appointments according to your needs. In the meantime, feel free to call me if you need any support. Good luck!"}], 'metadata': {'source': 'psych8k', 'type': 'single-turn', 'turn_count': 2, 'cognitive_distortions': ['emotional reasoning']}}
